In [1]:
import sys, os, pickle, time, re
from glob import glob
sys.path.append('../lib')
sys.path.append('../analysis')

In [2]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
from array import array

import ROOT as rt
rt.TGaxis.SetMaxDigits(4)
rt.gErrorIgnoreLevel = rt.kError
rt.RooMsgService.instance().setGlobalKillBelow(rt.RooFit.ERROR)
import ROOT.RooFit as rf
import root_numpy as rtnp

from analysis_utilities import drawOnCMSCanvas, getEff, DSetLoader
from histo_utilities import create_TH1D, create_TH2D, SetMaxToMaxHist, std_color_list
from gridVarQ2Plot import col_dic, label_dic
from progressBar import ProgressBar
from categoriesDef import categories
from pT_calibration_reader import pTCalReader

from lumi_utilities import getLumiByTrigger

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = 6*' ' + 'Simulation Internal'

donotdelete = []

Welcome to JupyROOT 6.12/07


In [3]:
cl = rt.TLine()
cl.SetLineColor(6)
cl.SetLineStyle(9)
cl.SetLineWidth(2)

In [ ]:
df = {}

dsMu = DSetLoader('B0_MuNuDmst_PUc0')

for cat in ['high']:
# for cat in ['low', 'mid', 'high']:
    ptCal[cat] = pTCalReader(calibration_file='../data/calibration/B0pTspectrum/pwWeights_{}.txt'.format(cat.capitalize()))
    
    fn = '/'+cat.capitalize()+'_corr.root'
    df['MC'+cat] = pd.DataFrame(rtnp.root2array(dsMu.skimmed_dir+fn))
    wPt['MC'+cat] = computeB0pTweights(df['MC'+cat], ptCal[cat])
    
    df['RD'+cat] = pd.DataFrame(rtnp.root2array('../data/cmsRD/skimmed/B2DstMu_B0_200515_'+cat.capitalize()+'_corr.root'))

    dsTau = DSetLoader('B0_TauNuDmst_PUc0')
    df['MCtau'+cat] = pd.DataFrame(rtnp.root2array(dsTau.skimmed_dir+'/High_corr.root'))
    wPt['MCtau'+cat] = computeB0pTweights(df['MCtau'+cat], ptCal[cat])

In [ ]:
s = '\n'.join([k for k in df['MChigh'].columns[1:]])
print(s)

In [ ]:
def make_comp_plot_1D(inputs, #[x, label, color=-1 (auto), weights (None)]
                      transformX = None,
                      binning=[None, None, None],
                      setLog = '',
                      axis_title = ['', ''],
                      scale_histo = 'norm',
                      opt='overflow+underflow',
                      leg_loc=[0.6, 0.7, 0.9, 0.93] # x0, y0, w, h
                     ): 
    
    if not leg_loc is None:
        leg = rt.TLegend(leg_loc[0], leg_loc[1], leg_loc[0] + leg_loc[2], leg_loc[1] + leg_loc[3])
        leg.SetBorderSize(0)
        leg.SetFillStyle(0)
        
    colors = []
    for p in inputs:
        if len(p) > 2:
            colors.append(p[2])
        else:
            colors.append(-1)
    jStd = 0
    for i,c in enumerate(colors):
        if c == -1:
            colors[i] = std_color_list[jStd]
            jStd += 1
    
    hList = []
    for i, p in enumerate(inputs):
        if transformX is None:
            x = p[0]
        else:
            x = transformX(p[0])
        label = p[1]
        w = p[3] if len(p) > 3 else None
        h = create_TH1D(x, weights=w,
                        binning=binning if i==0 else hList[0].binning, 
                        axis_title=axis_title,
                        scale_histo=scale_histo, opt=opt,
                       )
        h.SetLineColor(colors[i])
        if not leg_loc is None:
            leg.AddEntry(h, label, 'lp')
        hList.append(h)
            
    m = SetMaxToMaxHist(hList)
    c = drawOnCMSCanvas(CMS_lumi, hList, 'same')
    leg.Draw()
    
    if 'x' in setLog: c.SetLogx()
    if 'y' in setLog: c.SetLogy()
    
    c.dnd = [hList, leg]
    return c

# General plots

In [ ]:
txt = rt.TLatex()
txt.SetTextSize(0.04)
txt.SetTextFont(42)

In [ ]:
ctau = df['MChigh']['MC_B_ctau']
t0 = 0.457
t1 = 0.455 + 3*0.008
wCtau = t0/t1 * np.exp(ctau * (1/t0 - 1/t1))

In [ ]:
make_comp_plot_1D(
    [
        [df['MCtauhigh']['MC_B_ctau'], label_dic['tau']],
        [df['MChigh']['MC_B_ctau'], label_dic['mu']], 
        [df['MChigh']['MC_B_ctau'], label_dic['mu'] + '(+3#sigma)', -1, wCtau],
    ],
#     transformX = lambda x: 10*np.abs(x),
#     binning=np.logspace(-6, 1, 70), setLog='x',
    binning=[50, 0, 0.4],
    axis_title = ['B c#tau (gen truth) [cm]', 'Normalized events'],
    leg_loc=[0.6, 0.75, 0.28, 0.15]
)

txt.SetTextAlign(33)
txt.DrawLatexNDC(0.97, 0.94, 'High cat.')

In [ ]:
make_comp_plot_1D(
    [
        [df['MCtauhigh']['d_vtxD0pismu_PV'], label_dic['tau'] + '(pt w.)', -1, wPt['MCtauhigh'][0]],
        [df['MChigh']['d_vtxD0pismu_PV'], label_dic['mu'] + '(bare)'],
        [df['MChigh']['d_vtxD0pismu_PV'], label_dic['mu'] + '(pt w.)', -1, wPt['MChigh'][0]],
        [df['MChigh']['d_vtxD0pismu_PV'], label_dic['mu'] + '(pt w. & c#tau+3#sigma)', -1, wPt['MChigh'][0]*wCtau],
        [df['RDhigh']['d_vtxD0pismu_PV'], label_dic['data'], 1],
        
    ],
#     transformX = lambda x: 10*np.abs(x),
#     binning=np.logspace(-6, 1, 70), setLog='x',
    binning=[50, 0, 4],
    axis_title = ['|PV-SV| [cm]', 'Normalized events'],
    leg_loc=[0.6, 0.7, 0.35, 0.2]
)

txt.SetTextAlign(33)
txt.DrawLatexNDC(0.97, 0.94, 'High cat.')